# Importamos las bibliotecas necesarias para el ejercicio:

In [ ]:
import matplotlib.image as ima
import matplotlib.pyplot as plt     #Funciones para representación gráfica
import numpy as np                  #Funciones numéricas (arrays, matrices, etc.)
from skimage import color           #Funciones para cambiar a grises
from skimage import io              #Funciones para cambiar a grises
import cv2                          #Para segundo intento
from PIL import Image               #Para tercer intento
import pandas as pd
from numpy import asarray
import random
from sklearn.preprocessing import MinMaxScaler

# Cambio de formato. Imagen RGB - Escala de grises

In [ ]:
def vector_gris(Img_int):
    Array_image = np.array(Img_int)                                         #Se crea un array
    M_zeros = np.zeros((int(Array_image.shape[0]),int(Array_image.shape[1])))    # Crea array similar de ceros (filas,columnas)
    R = 0
    G = 0
    B = 0
    for n in range(Array_image.shape[0]):                                        #son 3 for por que es RGB entonces hay que convertir cada uno
        for m in range(Array_image.shape[1]):
            suma = 0
            for j in range(Array_image.shape[2]):
                if j==0:
                    R = Array_image[n,m,j]*0.299
                    suma = suma+R
                elif j==1:
                    G = Array_image[n,m,j]*0.587 #es G
                    suma = suma+G
                else:
                    B = Array_image[n,m,j]*0.114 #B
                    suma = suma+B
            M_zeros[n,m] = suma
    cv2.imwrite("imagenGris.jpg", M_zeros)                                       #Representa imagen, se escribe la matriz en una imagen (nombre de la imagen, que matriz)
    variable = cv2.imread("imagenGris.jpg")                                      #Leer la imagen que guardamos
    cv2.imshow('Hola', variable)                                                 #Mostrar la imagen que guardamos 
    Imagen_entrada=Image.open("imagenGris.jpg")
    x=np.array(Imagen_entrada)
    return(x)

def Scaler(Vector_gris):
 M_zeros = np.zeros((int(Vector_gris.shape[0]),int(Vector_gris.shape[1])))
 for i in range(Vector_gris.shape[0]):
  for j in range (Vector_gris.shape[1]):
    M_zeros[i,j] = np.round((np.array(((Vector_gris[i,j]/255)*2)-1)),1)
 return(M_zeros)

In [ ]:
Img_int = Image.open("manzana.jpeg")
Array_Int = vector_gris(Img_int)
print(Array_Int)
Valores_escalados = Scaler(Array_Int)
print(Valores_escalados)

# Primero, aquí definimos las funciones que se van a utilizar para procesar los arreglos, matrices, etc:

In [ ]:
#Cualquier matriz la coloca en forma de vector vertical
def ordenamiento(input_array):
  size            = (input_array.shape[0]*input_array.shape[1])
  tarray          = np.transpose(input_array)
  reordenamiento  = [np.reshape(tarray,size)]
  Matriz_entrenamiento  = np.transpose(reordenamiento)
  return(Matriz_entrenamiento)


#Inicio Random
def init_weights(input,output):
 W = np.zeros((int(input.shape[1]), int(output.shape[1])))
 for i in range(input.shape[0]):
   for j in range(output.shape[0]):
    W[i,j] = random.uniform(-1, 1)
 return(W)


#Calculo de ab: y=a=W*X  x=b=Vy  3x1   Multiplicacion normal
def funcion_activacion(W,input_matrix):
 tmp = np.matmul(W,input_matrix)
 result = np.zeros((int(tmp.shape[0]),int(tmp.shape[1])))
 d=0.1
 for i in range(tmp.shape[0]):
   for j in range(tmp.shape[1]):
    if tmp[i,j]>1:
      result[i,j]=1
    elif result[i,j]<-1:
      result[i,j]=-1
    else:
      result[i,j] = ((d+1)*(tmp[i,j]))-(d*((tmp[i,j])**3))
 return(result)


#FUNCION DE W ACTUAL
def update_weight(W_0,input_0,input,output_0,output):
 eta = 0.02
 p1 = np.matmul(input_0,output_0.T)
 p2 = np.matmul(input_0,output.T)
 p3 = np.matmul(input,output_0.T)
 p4 = np.matmul(input,output.T)
 error = p1+p2-p3-p4
 error = np.multiply(error,eta)
 W_new = np.add(W_0,error)
 return(W_new)


#LEARNING FUNCTION
def learn(W,V,x,y):

  #Update patterns
  x_act = funcion_activacion(V,y)
  y_act = funcion_activacion(W,x)
  
  i = 1

  #Here we compare manually if the both matrices have converged

  diff_x = np.subtract(x_act,x)
  diff_y = np.subtract(y_act,y)
  sum_x = np.sum(diff_x)
  sum_y = np.sum(diff_y)

  tol = 0.000000000001
  while( abs(sum_x) > tol and abs(sum_y) > tol ):
    
    print("\nit {}: \ndiff_x({}):\n{}\ndiff_y({}):\n{}\n".format(i,np.sum(diff_x),diff_x,np.sum(diff_y),diff_y))
    W = update_weight(W,y,y_act,x,x_act)
    V = update_weight(V,x,x_act,y,y_act)
    print("\n------- Computing new patterns ------------\n")
    y_act = funcion_activacion(W,x)
    x_act = funcion_activacion(V,y)
    print("new y:\n{}\n".format(y_act))
    print("new x:\n{}\n".format(x_act))
    diff_x = np.subtract(x_act,x)
    diff_y = np.subtract(y_act,y)
    sum_x = np.sum(diff_x)
    sum_y = np.sum(diff_y)
    #print("new W:\n{}\n".format(W))
    #print("new V:\n{}\n".format(V))
    i+=1
  
  return W,V

# Aquí se realiza el experimento de aprendizaje:

In [ ]:
input_noisy_x = ordenamiento(np.array([
                [1,0.8,0],
                [0,0,0],
                [0,0,0]
                ]))

tol = 32
print("Input:\n{}\n".format(input_noisy_x))
y_prev = np.round(funcion_activacion(W,input_noisy_x),decimals=tol)
print("y_prev:\n{}".format(y_prev))
converge = False
it = 1;
while(not converge):
#for i in range(10):
  new_x = np.round(funcion_activacion(V,y_prev),decimals=tol)
  y_act = np.round(funcion_activacion(W,new_x),decimals=tol)
  print("new_x:\n{}".format(new_x))
  print("y_act:\n{}".format(y_act))
  error = np.subtract(y_act,y_prev)
  print("Error({}):\n{}\n".format(np.sum(error),error))
  if abs(np.sum(error)) <= 1/(10**tol):
    converge = True
  y_prev = np.round(y_act,decimals=tol)
  it+=1

print("Recalled x:\n{}\n".format(np.round(new_x,decimals=tol)))
print("Recalled y:\n{}\n".format(np.round(y_act,decimals=tol)))



Input matrix:
\begin{matrix}
-1&0.6&0.8\\
0.2&1&-0.5\\
-0.7&0.6&1
\end{matrix}

Output matrix:
\begin{matrix}
-0.5&-0.4&0.8\\
-0.3&-0.7&1\\
0.3&0.9&1
\end{matrix}